In [1]:
from investment.optimization_model import *
from investment.data_loading import data_process_with_scenarios, load_all_weather_data
from investment.demand_model import demand_model
from investment.post_processing import load_classical_params
from investment.post_processing_heterogeneity import create_state_dataframe_heterogeneity, state_from_demand_trajectory_heterogeneity, process_files_from_folder_heterogeneity
import seaborn as sns
import datetime

import plotly.io as pio
import plotly.express as px
pio.renderers.default = 'colab'  # https://stackoverflow.com/a/65800700

# New simul (10/06)

## Single beta

Here, I test the new code with a single possible value for beta. The goal is to observe 1) if the code works, 2) the impact of changing the devaluation calculation in the code.

Conclusions: code works as expected, we observe less investment, which is logical since the change in the code leads to less profits.

In [2]:
directory = "outputs/1006_singlebeta_no_demand_uncertainty/"
kwd1 = "weight"
kwd2 = "availnuc"

additional_parameters, trans_matrix, demand_states, joint_law, x_cutoff, y_cutoff, Q_offshore, weather_params = load_classical_params()

state_distribution_dict, objective_gap_dict, index_objective_dict, list_beta_dict, list_weight_beta_dict = process_files_from_folder_heterogeneity(directory, kwd1, kwd2)

demand_trajectory_low = [1, 0, 0, 0, 0]
demand_trajectory_middle = [1, 1, 1, 1, 1]
demand_trajectory_high = [1, 2, 2, 2, 2]
demand_trajectory_middle_high = [1, 1, 1, 2, 2]

demand_trajectory_dict = {
    "low": demand_trajectory_low,
    "middle": demand_trajectory_middle,
    "high": demand_trajectory_high}

list_beta, list_weight = [0.6], [1]
time_list = ['2025', '2030', '2035', '2040', '2045']
state_dataframe = create_state_dataframe_heterogeneity(state_distribution_dict, demand_trajectory_dict, demand_list=list(demand_trajectory_dict.keys()), keyword1=kwd1,
                                                keyword2=kwd2, param_list=list(state_distribution_dict.keys()), time_list=time_list, list_beta=list_beta,
                                                      list_weight=list_weight)


weightsingleton_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/1006_singlebeta_no_demand_uncertainty/weightsingleton_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0/13-58-50_beta_discretization.npz'

In [4]:
tmp = state_dataframe.loc[(state_dataframe.tec == 'sun') ]
p = plot_state_trajectory_curve_2param(tmp, static=False, keyword1=kwd1, keyword2=kwd2)


## Multiple beta

In [3]:
directory = "outputs/1006_multiplebeta_no_demand_uncertainty/"
kwd1 = "weight"
kwd2 = "availnuc"

additional_parameters, trans_matrix, demand_states, joint_law, x_cutoff, y_cutoff, Q_offshore, weather_params = load_classical_params()

state_distribution_dict, objective_gap_dict, index_objective_dict, list_beta_dict, list_weight_beta_dict = process_files_from_folder_heterogeneity(directory, kwd1, kwd2)

demand_trajectory_low = [1, 0, 0, 0, 0]
demand_trajectory_middle = [1, 1, 1, 1, 1]
demand_trajectory_high = [1, 2, 2, 2, 2]
demand_trajectory_middle_high = [1, 1, 1, 2, 2]

demand_trajectory_dict = {
    "low": demand_trajectory_low,
    "middle": demand_trajectory_middle,
    "high": demand_trajectory_high}

time_list = ['2025', '2030', '2035', '2040', '2045']
state_dataframe = create_state_dataframe_heterogeneity(state_distribution_dict, list_beta_dict, list_weight_beta_dict, demand_trajectory_dict,
                                                       demand_list=list(demand_trajectory_dict.keys()), keyword1=kwd1,
                                                        keyword2=kwd2, param_list=list(state_distribution_dict.keys()), time_list=time_list)


weightbinom_04_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weightbinom_04_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightbinom_06_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weightbinom_06_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightnormal_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weightnormal_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightuniform_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weightuniform_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0


In [4]:
state_dataframe

,time,state,tec,weight,availnuc,demand_trajectory,beta,beta_weight
0,2025,9.762642,sun,binom,0p8,low,0.0,0.000105
1,2025,10.285952,sun,binom,0p8,low,0.1,0.001573
2,2025,10.809262,sun,binom,0p8,low,0.2,0.010617
3,2025,11.332572,sun,binom,0p8,low,0.3,0.042467
4,2025,11.855882,sun,binom,0p8,low,0.4,0.111477
...,...,...,...,...,...,...,...,...
1975,2045,136.930198,wind,uniform,0p9,high,0.6,0.090909
1976,2045,138.846061,wind,uniform,0p9,high,0.7,0.090909
1977,2045,140.761925,wind,uniform,0p9,high,0.8,0.090909
1978,2045,142.677788,wind,uniform,0p9,high,0.9,0.090909


In [5]:
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=tmp.loc[x.index, "beta_weight"])

tmp = state_dataframe.loc[(state_dataframe.demand_trajectory == 'middle') & (state_dataframe.availnuc == '0p8')]
tmp = tmp.groupby(["time", "tec", "weight"]).agg(average_state=("state", wm), average_beta=("beta", wm))
tmp = tmp.reset_index()
tmp

,time,tec,weight,average_state,average_beta
0,2025,sun,binom,12.902502,0.6
1,2025,sun,normal,12.644990,0.5
2,2025,sun,uniform,12.644989,0.5
3,2025,wind,binom,21.134474,0.6
4,2025,wind,normal,20.896032,0.5
5,2025,wind,uniform,20.896029,0.5
6,2030,sun,binom,30.773164,0.6
7,2030,sun,normal,30.249133,0.5
8,2030,sun,uniform,30.249132,0.5
9,2030,wind,binom,49.540579,0.6


In [6]:
px.line(tmp, x="time", y="average_state", line_dash="weight", facet_col="tec")

In [38]:
tmp.loc[(tmp.time == "2025") & (tmp.tec == "sun") & ((tmp.weight == "normal") | (tmp.weight == 'uniform'))]

,time,state,tec,weight,beta,beta_weight,weighted_state
770,2025,9.980589,sun,normal,0.0,0.010000,0.099806
771,2025,10.513469,sun,normal,0.1,0.010000,0.105135
772,2025,11.046349,sun,normal,0.2,0.030000,0.331390
773,2025,11.579229,sun,normal,0.3,0.100000,1.157923
774,2025,12.112110,sun,normal,0.4,0.200000,2.422422
775,2025,12.644990,sun,normal,0.5,0.300000,3.793497
776,2025,13.177870,sun,normal,0.6,0.200000,2.635574
777,2025,13.710750,sun,normal,0.7,0.100000,1.371075
778,2025,14.243630,sun,normal,0.8,0.030000,0.427309
779,2025,14.776510,sun,normal,0.9,0.010000,0.147765


In [7]:
px.scatter(tmp.loc[(tmp.time == "2025") & (tmp.weight == "normal") & (tmp.tec == "sun")], x='state', y='beta_weight')

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['time', 'tec', 'weight', 'average_state', 'average_beta'] but received: state

In [40]:
px.scatter(tmp.loc[(tmp.time == "2030") & (tmp.weight == "normal") & (tmp.tec == "sun")], x='beta', y='state')

In [41]:
px.scatter(tmp.loc[(tmp.time == "2040") & (tmp.weight == "normal") & (tmp.tec == "sun")], x='state', y='beta_weight')

In [ ]:
tmp.groupby(['time', 'tec', 'weight'])

In [7]:
state_dataframe

,time,state,tec,weight,availnuc,demand_trajectory,beta
0,2025,9.762642,sun,binom,0p8,low,0.0
1,2025,10.285952,sun,binom,0p8,low,0.1
2,2025,10.809262,sun,binom,0p8,low,0.2
3,2025,11.332572,sun,binom,0p8,low,0.3
4,2025,11.855882,sun,binom,0p8,low,0.4
...,...,...,...,...,...,...,...
1975,2045,136.930198,wind,uniform,0p9,high,0.6
1976,2045,138.846061,wind,uniform,0p9,high,0.7
1977,2045,140.761925,wind,uniform,0p9,high,0.8
1978,2045,142.677788,wind,uniform,0p9,high,0.9


## Distribution beta (10/7)

In [19]:
directory = "outputs/1007_beta_heterogeneity_no_demand_uncertainty/"
kwd1 = "weight"
kwd2 = "availnuc"

additional_parameters, trans_matrix, demand_states, joint_law, x_cutoff, y_cutoff, Q_offshore, weather_params = load_classical_params()

state_distribution_dict, objective_gap_dict, index_objective_dict, list_beta_dict, list_weight_beta_dict = process_files_from_folder_heterogeneity(directory, kwd1, kwd2)

demand_trajectory_low = [1, 0, 0, 0, 0]
demand_trajectory_middle = [1, 1, 1, 1, 1]
demand_trajectory_high = [1, 2, 2, 2, 2]
demand_trajectory_middle_high = [1, 1, 1, 2, 2]

demand_trajectory_dict = {
    "low": demand_trajectory_low,
    "middle": demand_trajectory_middle,
    "high": demand_trajectory_high}

time_list = ['2025', '2030', '2035', '2040', '2045']
state_dataframe = create_state_dataframe_heterogeneity(state_distribution_dict, list_beta_dict, list_weight_beta_dict, demand_trajectory_dict,
                                                       demand_list=list(demand_trajectory_dict.keys()), keyword1=kwd1,
                                                        keyword2=kwd2, param_list=list(state_distribution_dict.keys()), time_list=time_list)


weightbinom04_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weightbinom04_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightbinom06_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weightbinom06_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightbinom08_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weightbinom08_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weighthome1_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weighthome1_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weighthome2_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weighthome2_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weighthome3_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weighthome3_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightsingleton05_discount0p04_nu0p15_premium0p0_availnuc0p8_cvar0p05_cap10000p0
weights

In [20]:
state_dataframe

,time,state,tec,weight,availnuc,demand_trajectory,beta,beta_weight
0,2025,10.219563,sun,binom04,0p8,low,0.0,0.006047
1,2025,10.764500,sun,binom04,0p8,low,0.1,0.040311
2,2025,11.309437,sun,binom04,0p8,low,0.2,0.120932
3,2025,11.854374,sun,binom04,0p8,low,0.3,0.214991
4,2025,12.399311,sun,binom04,0p8,low,0.4,0.250823
...,...,...,...,...,...,...,...,...
5935,2045,136.930198,wind,uniform,0p9,high,0.6,0.090909
5936,2045,138.846061,wind,uniform,0p9,high,0.7,0.090909
5937,2045,140.761925,wind,uniform,0p9,high,0.8,0.090909
5938,2045,142.677788,wind,uniform,0p9,high,0.9,0.090909


In [14]:
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=tmp.loc[x.index, "beta_weight"])

tmp = state_dataframe.loc[(state_dataframe.demand_trajectory == 'middle') & (state_dataframe.availnuc == '0p8')]
tmp = tmp.groupby(["time", "tec", "weight"]).agg(average_state=("state", wm), average_beta=("beta", wm))
tmp = tmp.reset_index()
tmp

,time,tec,weight,average_state,average_beta
0,2025,sun,binom04,12.399311,0.4
1,2025,sun,binom06,12.902502,0.6
2,2025,sun,binom08,13.366957,0.8
3,2025,sun,home1,12.644990,0.5
4,2025,sun,home2,12.644990,0.5
...,...,...,...,...,...
85,2045,wind,home2,157.347312,0.5
86,2045,wind,home3,157.347405,0.5
87,2045,wind,singleton05,157.347309,0.5
88,2045,wind,singleton08,159.788296,0.8


In [16]:
px.line(tmp, x="time", y="average_state", color="weight", facet_col="tec")

# Gamma heterogeneity

In [4]:
directory = "outputs/1014_gamma_heterogeneity_no_demand_uncertainty/"
kwd1 = "weight"
kwd2 = "beta"
name_param = "gamma"

additional_parameters, trans_matrix, demand_states, joint_law, x_cutoff, y_cutoff, Q_offshore, weather_params = load_classical_params()

state_distribution_dict, objective_gap_dict, index_objective_dict, list_gamma_dict, list_weight_gamma_dict = process_files_from_folder_heterogeneity(directory, kwd1, kwd2)

demand_trajectory_low = [1, 0, 0, 0, 0]
demand_trajectory_middle = [1, 1, 1, 1, 1]
demand_trajectory_high = [1, 2, 2, 2, 2]
demand_trajectory_middle_high = [1, 1, 1, 2, 2]

demand_trajectory_dict = {
    "low": demand_trajectory_low,
    "middle": demand_trajectory_middle,
    "high": demand_trajectory_high}

time_list = ['2025', '2030', '2035', '2040', '2045']
state_dataframe = create_state_dataframe_heterogeneity(state_distribution_dict, list_gamma_dict, list_weight_gamma_dict, name_param=name_param, 
                                                       demand_trajectory_dict=demand_trajectory_dict, demand_list=list(demand_trajectory_dict.keys()), keyword1=kwd1,
                                                        keyword2=kwd2, param_list=list(state_distribution_dict.keys()), time_list=time_list)


weight2dirac_beta0p4_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weight2dirac_beta0p8_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weight3dirac_beta0p4_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weight3dirac_beta0p8_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightsingleton0p92_beta0p4_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightsingleton0p92_beta0p8_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightsingleton0p94_beta0p4_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightsingleton0p94_beta0p8_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightsingleton0p96_beta0p4_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightsingleton0p96_beta0p8_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightsingleton0p98_beta0p4_discount0p04_nu0p15_premium0p0_availnuc0p9_cvar0p05_cap10000p0
weightsingleton0p98_beta0p8

In [3]:
state_dataframe

,time,state,tec,weight,beta,demand_trajectory,gamma,gamma_weight
0,2025,7.303272,sun,2dirac,0p4,low,0.9,0.5
1,2025,10.037026,sun,2dirac,0p4,low,1.1,0.5
2,2030,18.480715,sun,2dirac,0p4,low,0.9,0.5
3,2030,24.549963,sun,2dirac,0p4,low,1.1,0.5
4,2035,34.240210,sun,2dirac,0p4,low,0.9,0.5
...,...,...,...,...,...,...,...,...
955,2025,14.375558,wind,singleton1p1,0p8,high,1.1,1.0
956,2030,34.459306,wind,singleton1p1,0p8,high,1.1,1.0
957,2035,62.077573,wind,singleton1p1,0p8,high,1.1,1.0
958,2040,93.854493,wind,singleton1p1,0p8,high,1.1,1.0


## Distribution spread

### Sun

In [14]:
state_dataframe.loc[(state_dataframe.tec == 'sun') & (state_dataframe.weight == "2dirac") & (state_dataframe.demand_trajectory == 'low') & (state_dataframe.beta == '0p8')]

,time,state,tec,weight,beta,demand_trajectory,gamma,gamma_weight
60,2025,7.856901,sun,2dirac,0p8,low,0.9,0.5
61,2025,10.713684,sun,2dirac,0p8,low,1.1,0.5
62,2030,19.707604,sun,2dirac,0p8,low,0.9,0.5
63,2030,26.049494,sun,2dirac,0p8,low,1.1,0.5
64,2035,36.117222,sun,2dirac,0p8,low,0.9,0.5
65,2035,46.774027,sun,2dirac,0p8,low,1.1,0.5
66,2040,55.287774,sun,2dirac,0p8,low,0.9,0.5
67,2040,70.735760,sun,2dirac,0p8,low,1.1,0.5
68,2045,74.531962,sun,2dirac,0p8,low,0.9,0.5
69,2045,94.707832,sun,2dirac,0p8,low,1.1,0.5


In [12]:
state_dataframe.loc[(state_dataframe.tec == 'sun') & ((state_dataframe.weight == "singleton0p9") | (state_dataframe.weight == 'singleton1p1')) & (state_dataframe.demand_trajectory == 'low') & (state_dataframe.beta == '0p8')]
                                                                                                    

,time,state,tec,weight,beta,demand_trajectory,gamma,gamma_weight
570,2025,10.530525,sun,singleton0p9,0p8,low,0.9,1.0
571,2030,25.818511,sun,singleton0p9,0p8,low,0.9,1.0
572,2035,46.703942,sun,singleton0p9,0p8,low,0.9,1.0
573,2040,71.028433,sun,singleton0p9,0p8,low,0.9,1.0
574,2045,95.445415,sun,singleton0p9,0p8,low,0.9,1.0
930,2025,9.408719,sun,singleton1p1,0p8,low,1.1,1.0
931,2030,23.107911,sun,singleton1p1,0p8,low,1.1,1.0
932,2035,41.728116,sun,singleton1p1,0p8,low,1.1,1.0
933,2040,63.271733,sun,singleton1p1,0p8,low,1.1,1.0
934,2045,84.863878,sun,singleton1p1,0p8,low,1.1,1.0


### Wind

In [20]:
state_dataframe.loc[(state_dataframe.tec == 'wind') & (state_dataframe.weight == "2dirac") & (state_dataframe.demand_trajectory == 'low') & (state_dataframe.beta == '0p8')]

,time,state,tec,weight,beta,demand_trajectory,gamma,gamma_weight
70,2025,7.623545,wind,2dirac,0p8,low,0.9,0.5
71,2025,10.474754,wind,2dirac,0p8,low,1.1,0.5
72,2030,19.182426,wind,2dirac,0p8,low,0.9,0.5
73,2030,25.539518,wind,2dirac,0p8,low,1.1,0.5
74,2035,35.769464,wind,2dirac,0p8,low,0.9,0.5
75,2035,46.525909,wind,2dirac,0p8,low,1.1,0.5
76,2040,55.330002,wind,2dirac,0p8,low,0.9,0.5
77,2040,70.965523,wind,2dirac,0p8,low,1.1,0.5
78,2045,74.522454,wind,2dirac,0p8,low,0.9,0.5
79,2045,94.829127,wind,2dirac,0p8,low,1.1,0.5


In [19]:
state_dataframe.loc[(state_dataframe.tec == 'wind') & ((state_dataframe.weight == "singleton0p9") | (state_dataframe.weight == 'singleton1p1')) & (state_dataframe.demand_trajectory == 'low') & (state_dataframe.beta == '0p8')]
                                                                                                    

,time,state,tec,weight,beta,demand_trajectory,gamma,gamma_weight
575,2025,17.507878,wind,singleton0p9,0p8,low,0.9,1.0
576,2030,41.780468,wind,singleton0p9,0p8,low,0.9,1.0
577,2035,74.995081,wind,singleton0p9,0p8,low,0.9,1.0
578,2040,113.119885,wind,singleton0p9,0p8,low,0.9,1.0
579,2045,150.090365,wind,singleton0p9,0p8,low,0.9,1.0
935,2025,14.375558,wind,singleton1p1,0p8,low,1.1,1.0
936,2030,34.459306,wind,singleton1p1,0p8,low,1.1,1.0
937,2035,62.077573,wind,singleton1p1,0p8,low,1.1,1.0
938,2040,93.854493,wind,singleton1p1,0p8,low,1.1,1.0
939,2045,124.508838,wind,singleton1p1,0p8,low,1.1,1.0


## Beta = 0.8

In [6]:
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=tmp.loc[x.index, f"{name_param}_weight"])

tmp = state_dataframe.loc[(state_dataframe.demand_trajectory == 'middle') & (state_dataframe.beta == '0p8')]
tmp = tmp.groupby(["time", "tec", "weight"]).agg(average_state=("state", wm), average_beta=(name_param, wm))
tmp = tmp.reset_index()
tmp

,time,tec,weight,average_state,average_beta
0,2025,sun,2dirac,9.285292,1.00
1,2025,sun,3dirac,8.786278,1.00
2,2025,sun,singleton0p9,10.530525,0.90
3,2025,sun,singleton0p92,10.418861,0.92
4,2025,sun,singleton0p94,10.301842,0.94
...,...,...,...,...,...
125,2045,wind,singleton1p02,134.742504,1.02
126,2045,wind,singleton1p04,132.194643,1.04
127,2045,wind,singleton1p06,129.381168,1.06
128,2045,wind,singleton1p08,127.196764,1.08


In [21]:
px.line(tmp, x="time", y="average_state", color="weight", facet_col="tec")

## Beta = 0.4

In [15]:
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=tmp.loc[x.index, f"{name_param}_weight"])

tmp = state_dataframe.loc[(state_dataframe.demand_trajectory == 'middle') & (state_dataframe.beta == '0p4')]
tmp = tmp.groupby(["time", "tec", "weight"]).agg(average_state=("state", wm), average_beta=(name_param, wm))
tmp = tmp.reset_index()
tmp

,time,tec,weight,average_state,average_beta
0,2025,sun,2dirac,8.670149,1.00
1,2025,sun,3dirac,8.184189,1.00
2,2025,sun,singleton0p9,9.887040,0.90
3,2025,sun,singleton0p92,9.781743,0.92
4,2025,sun,singleton0p94,9.650054,0.94
...,...,...,...,...,...
125,2045,wind,singleton1p02,132.337380,1.02
126,2045,wind,singleton1p04,130.012309,1.04
127,2045,wind,singleton1p06,127.428391,1.06
128,2045,wind,singleton1p08,124.636270,1.08


In [16]:
px.line(tmp, x="time", y="average_state", color="weight", facet_col="tec")

# Useful tools

Remarque préalable importante:
1. Il faut faire attention au choix du price cap
2. Le state détermine si on regarde le scénario moyen, ou le scénario CVAR
3. Le choix des valeurs d'investissement sont importantes. Actuellement, il faut les mettre à jour pour les nouvelles valeurs d'investissement avec les nouveaux scénarios. Sinon, on a des valeurs trop élevées qui biaisent les prix observés menant aux profits des renouvelables, et donc aux décisions d'investissement.
4. Il faut bien aligner les valeurs d'investissement et l'instant t considéré

In [19]:
price_cap = 10000  ## ATTENTION price cap !!
avail_nuc = 0.8  ## ATTENTION avail nuc !!
imports = True
flex = "high"

additional_parameters, trans_matrix, demand_states, joint_law, x_cutoff, y_cutoff, Q_offshore, weather_params = load_classical_params()
additional_parameters["market_price"] = price_cap  

# Weather params
joint_law, x_cutoff, y_cutoff, Q_offshore = data_process_with_scenarios(period="large", avail_nuclear=avail_nuc, imports=imports, flex=flex)
demand_centered = np.array(joint_law.demand_centered_group)
pv = np.array(joint_law.pv_group)
onshore = np.array(joint_law.onshore_group)
offshore = np.array(joint_law.offshore_group)
river = np.array(joint_law.river_group)
hydro_prod = np.array(joint_law.hydro_prod_group)
probability = np.array(joint_law.probability)
weather_params = {
    "demand": demand_centered,
    "sun": pv,
    "onshore": onshore,
    "offshore": offshore,
    "river": river,
    "hydro": hydro_prod,
    "proba": probability
}

demand_centered_tot, pv_tot, onshore_tot, offshore_tot, river_tot, hydro_prod_tot, probability_tot, year_tot = load_all_weather_data(
    1980, 2019, imports=imports, flex=flex)
weather_tot_params = {
    "demand": demand_centered_tot,
    "sun": pv_tot,
    "onshore": onshore_tot,
    "offshore": offshore_tot,
    "river": river_tot,
    "hydro": hydro_prod_tot,
    "proba": probability_tot,
    "years": year_tot
}

t=2
state = 1  # to change to look at CVAR
Q_offshore_t, x_cutoff_t, y_cutoff_t = Q_offshore[t][state], x_cutoff[t][state], y_cutoff[t][state]
D_average_t = demand_states[t,:][state]
# Q_sun, Q_wind = 44, 64

# New mix values, change in price function
# Q_sun, Q_wind = 37.24, 52.9  # new simul with new scenario mix, availnuc = 1, beta=0.5, cap = 10000
# Q_sun, Q_wind = 38.5, 54  # new simul 08/31 with new scenario mix, availnuc = 1, beta=0.5, cap = 10000, and change in price function

# New mix values, change in price function, adding imports
Q_sun, Q_wind = 55.8, 85  # new simul with new scenario mix, availnuc = 0.8, beta=0.2, cap = 10000
Q_sun, Q_wind = 68.5, 106.5  # new simul 09/02 with new scenario mix, availnuc = 0.8, beta=0.2, cap = 10000, change in price function and imports

# New mix values, change in price function, adding imports, t=2, beta=0.2
# Q_sun, Q_wind = 51.6, 79  # new simul with new scenario mix, availnuc = 0.8, beta=0.2, cap = 10000
Q_sun, Q_wind = 68.5, 106.5  # new simul 09/02 with new scenario mix, availnuc = 0.8, beta=0.2, cap = 10000, change in price function and imports, low
Q_sun, Q_wind = 56.7, 85.8  # new simul 09/02 with new scenario mix, availnuc = 0.8, beta=0.2, cap = 10000, change in price function and imports, high

# New mix values, change in price function, adding imports, t=2, beta=1
# Q_sun, Q_wind = 40, 58  # new simul with new scenario mix, availnuc = 1, beta=1, cap = 10000
# Q_sun, Q_wind = 55, 81  # new simul 09/02 with new scenario mix, availnuc = 1, beta=1, cap = 10000, change in price function and imports

# New mix values, change in price function, adding imports, t=0
# Q_sun, Q_wind = 12, 19.26  # new simul with new scenario mix, availnuc = 0.8, beta=0.2, cap = 10000
# Q_sun, Q_wind = 17.2, 27.8  # new simul 09/02 with new scenario mix, availnuc = 0.8, beta=0.2, cap = 10000, change in price function and imports

# Q_sun, Q_wind = 55, 85  # new simul with new scenario mix, availnuc = 0.8, beta=0.5, cap = 10000
# Q_sun, Q_wind = 37, 48  # price cap à 3000

In [20]:
D_average_t

65.9

## Statistics on price

In [21]:
y = 1987
mask = (year_tot == y)
weather_params_years = {
    "demand": demand_centered_tot[mask],
    "sun": pv_tot[mask],
    "onshore": onshore_tot[mask],
    "offshore": offshore_tot[mask],
    "river": river_tot[mask],
    "hydro": hydro_prod_tot[mask],
    "proba": probability_tot[mask]
    }
weather_price_dataframe_year = create_hourly_dataframe(D_average_t, Q_sun, Q_wind, Q_offshore_t, x_cutoff_t,
                                                  y_cutoff_t, weather_params=weather_params_years, add_params=additional_parameters)
weather_price_dataframe_year = get_price_duration_curve(weather_price_dataframe_year, multiple=False)

In [22]:
# Explore prices / some statistics
print(f"Some statistics for year {y}... \n")

# Average price reached by renewables
print("Average price")
avg_price = (weather_price_dataframe_year.price * weather_price_dataframe_year.gamma_sun).sum() / weather_price_dataframe_year.gamma_sun.sum()
print(f'Average price reached by renewables: {avg_price: .3f} EUR/MWH')

tmp = weather_price_dataframe_year.loc[weather_price_dataframe_year.price < price_cap]
avg_price_without_cap = (tmp.price * tmp.gamma_sun).sum() / tmp.gamma_sun.sum()
print(f'Average price reached by renewables without the price cap: {avg_price_without_cap: .3f} EUR/MWH \n')

# Impact of price cap
print("Impact of price cap")
tmp_pricecap = weather_price_dataframe_year.loc[weather_price_dataframe_year.price == price_cap]
avg_gamma_pricecap = (tmp_pricecap["gamma_sun"] * tmp_pricecap["proba"]).sum()/tmp_pricecap["proba"].sum()
print(f'Average capacity factor of sun when price cap is reached: {avg_gamma_pricecap: .5f}')
print(f'Probability of reaching price cap: {tmp_pricecap["proba"].sum(): .5f} \n')

# Capacity factor
print("Capacity factor")
avg_gamma = weather_price_dataframe_year.gamma_sun.mean()
avg_gamma_lowprice = weather_price_dataframe_year.loc[weather_price_dataframe_year.price < 13].gamma_sun.mean()
avg_gamma_highprice = weather_price_dataframe_year.loc[weather_price_dataframe_year.price > 13].gamma_sun.mean()
print(f"Average capacity factor for sun: {avg_gamma: .5f}")
print(f"Average capacity factor for sun when prices are low: {avg_gamma_lowprice: .5f}")
print(f"Average capacity factor for sun when prices are high: {avg_gamma_highprice: .5f}")


Some statistics for year 1987... 

Average price
Average price reached by renewables:  202.028 EUR/MWH
Average price reached by renewables without the price cap:  6.766 EUR/MWH 

Impact of price cap
Average capacity factor of sun when price cap is reached:  0.01846
Probability of reaching price cap:  0.10639 

Capacity factor
Average capacity factor for sun:  0.14937
Average capacity factor for sun when prices are low:  0.19257
Average capacity factor for sun when prices are high:  0.05215


In [277]:
# Explore prices / some statistics
print("Some statistics... \n")

# Average price reached by renewables
print("Average price")
avg_price = (weather_price_dataframe_year.price * weather_price_dataframe_year.gamma_onshore).sum() / weather_price_dataframe_year.gamma_onshore.sum()
print(f'Average price reached by wind renewables: {avg_price: .3f} EUR/MWH')

tmp = weather_price_dataframe_year.loc[weather_price_dataframe_year.price < price_cap]
avg_price_without_cap = (tmp.price * tmp.gamma_onshore).sum() / tmp.gamma_onshore.sum()
print(f'Average price reached by wind renewables without the price cap: {avg_price_without_cap: .3f} EUR/MWH \n')

# Impact of price cap
print("Impact of price cap")
tmp_pricecap = weather_price_dataframe_year.loc[weather_price_dataframe_year.price == price_cap]
avg_gamma_pricecap = (tmp_pricecap["gamma_onshore"] * tmp_pricecap["proba"]).sum()/tmp_pricecap["proba"].sum()
print(f'Average capacity factor of wind when price cap is reached: {avg_gamma_pricecap: .5f}')
print(f'Probability of reaching price cap: {tmp_pricecap["proba"].sum(): .5f} \n')

# Capacity factor
print("Capacity factor")
avg_gamma = weather_price_dataframe_year.gamma_onshore.mean()
avg_gamma_lowprice = weather_price_dataframe_year.loc[weather_price_dataframe_year.price < 13].gamma_onshore.mean()
avg_gamma_highprice = weather_price_dataframe_year.loc[weather_price_dataframe_year.price > 13].gamma_onshore.mean()
print(f"Average capacity factor for wind: {avg_gamma: .5f}")
print(f"Average capacity factor for wind when prices are low: {avg_gamma_lowprice: .5f}")
print(f"Average capacity factor for wind when prices are high: {avg_gamma_highprice: .5f}")



Some statistics... 

Average price
Average price reached by wind renewables:  61.732 EUR/MWH
Average price reached by wind renewables without the price cap:  8.958 EUR/MWH 

Impact of price cap
Average capacity factor of wind when price cap is reached:  0.03687
Probability of reaching price cap:  0.02209 

Capacity factor
Average capacity factor for wind:  0.18404
Average capacity factor for wind when prices are low:  0.20213
Average capacity factor for wind when prices are high:  0.10066


In [214]:
tmp_pricecap = weather_price_dataframe_year.loc[weather_price_dataframe_year.price == price_cap]
(tmp_pricecap["gamma_sun"] * tmp_pricecap["proba"]).sum()/tmp_pricecap["proba"].sum()

0.016020833333329952

In [215]:
tmp_pricecap["proba"].sum()

0.0109289617486312

## Price duration curve

In [90]:
mask = (year_tot == 1987)
weather_params_years = {
    "demand": demand_centered_tot[mask],
    "sun": pv_tot[mask],
    "onshore": onshore_tot[mask],
    "offshore": offshore_tot[mask],
    "river": river_tot[mask],
    "hydro": hydro_prod_tot[mask],
    "proba": probability_tot[mask]
    }
weather_price_dataframe_year = create_hourly_dataframe(D_average_t, Q_sun, Q_wind, Q_offshore_t, x_cutoff_t,
                                                  y_cutoff_t, weather_params=weather_params_years, add_params=additional_parameters)
weather_price_dataframe_year = get_price_duration_curve(weather_price_dataframe_year, multiple=False)

In [91]:
p = plot_price_duration_curve(weather_price_dataframe_year, market_price_cap=additional_parameters['market_price'], cap=True, static=False, multiple=False,
                              keyword="", show=True)

In [201]:
# 2016
p = plot_price_duration_curve(weather_price_dataframe_year, market_price_cap=additional_parameters['market_price'], cap=True, static=False, multiple=False,
                              keyword="", show=True)

In [ ]:
weather_price_dataframe_tot = create_hourly_dataframe_2param([1, 1, 1, 1], state_distribution_dict, demand_states, x_cutoff, y_cutoff,
                                     Q_offshore, weather_params, additional_parameters, keyword1=kwd1, keyword2=kwd2, list_param=list(state_distribution_dict.keys()))
weather_price_dataframe_tot = get_price_duration_curve_2param(weather_price_dataframe_tot, keyword1=kwd1, keyword2=kwd2)

# Joint law (checks)

I want to check that the new code captures correctly joint_law, as it was done previously

In [77]:
joint_law_old = pd.read_csv("inputs/joint_law/joint_law_2015_2019.csv", index_col=0)
joint_law_new = pd.read_csv("inputs/joint_law/joint_law_2015_2019_test.csv", index_col=0)

In [79]:
joint_law_old.sort_values(by='probability', ascending=False).iloc[0:10]

,pv_group,onshore_group,offshore_group,river_group,hydro_prod_group,demand_centered_group,occurrence,probability
1418,0.0,0.1613,0.2503,0.000,-2.82,-23.561,121,0.002761
2734,0.0,0.2852,0.5279,0.000,-2.82,-23.561,112,0.002556
216,0.0,0.0700,0.0000,0.000,-2.82,-23.561,106,0.002419
515,0.0,0.1013,0.0000,0.000,-2.82,-23.561,101,0.002305
3518,0.0,0.4809,0.8491,0.522,-2.82,5.533,95,0.002168
1029,0.0,0.1302,0.2503,0.000,-2.82,-23.561,95,0.002168
3517,0.0,0.4809,0.8491,0.522,-2.82,1.234,91,0.002076
3492,0.0,0.4809,0.8491,0.354,-2.82,5.533,89,0.002031
3491,0.0,0.4809,0.8491,0.354,-2.82,1.234,85,0.001940
1808,0.0,0.1963,0.2503,0.000,-2.82,-23.561,76,0.001734


In [80]:
joint_law_new.sort_values(by='probability', ascending=False).iloc[0:10]

,pv_group,onshore_group,offshore_group,river_group,hydro_prod_group,demand_centered_group,occurrence,probability
1418,0.0,0.1613,0.2503,0.000,-2.819,-23.560,121,0.002761
2734,0.0,0.2852,0.5279,0.000,-2.819,-23.560,112,0.002556
216,0.0,0.0700,0.0000,0.000,-2.819,-23.560,106,0.002419
515,0.0,0.1013,0.0000,0.000,-2.819,-23.560,101,0.002305
3518,0.0,0.4809,0.8491,0.522,-2.819,5.533,95,0.002168
1029,0.0,0.1302,0.2503,0.000,-2.819,-23.560,95,0.002168
3517,0.0,0.4809,0.8491,0.522,-2.819,1.233,91,0.002076
3492,0.0,0.4809,0.8491,0.354,-2.819,5.533,89,0.002031
3491,0.0,0.4809,0.8491,0.354,-2.819,1.233,85,0.001940
1808,0.0,0.1963,0.2503,0.000,-2.819,-23.560,76,0.001734
